In [0]:
## New
!pip install syft


In [0]:

import torch
import json
import numpy as np
from PIL import Image
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
import matplotlib.pyplot as plt
from collections import OrderedDict
from torchvision import transforms, models, datasets


from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '1YjixsAkG1w4j13Bc9a0WqTVp6t3jCR7e'})
download.GetContentFile('full_data.zip')

In [0]:
#get_ipython().system_raw("unrar x full_data.rar")
!unzip full_data.zip

In [0]:
# Directory for our dataset
data_dir  = "full_data"
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [0]:
import syft as sy
hook = sy.TorchHook(torch)
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [0]:
## New
class Arguments():
    def __init__(self):
        self.batch_size = 224
        self.test_batch_size = 32
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = True

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [0]:
## New ##
# Defining transforms
train_transforms = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                        [0.229, 0.224, 0.225])
])

validation_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
        
                         [0.229, 0.224, 0.225])
])


In [0]:
class CustomImageFolder(ImageFolder):
    def __init__(self, root, transform=None, target_transform=None):
        super(CustomImageFolder, self).__init__(str(root), transform, target_transform) 
        self.data = self.imgs



train_dataset = CustomImageFolder(train_dir, transform=train_transforms)

federated_train_dataset = train_dataset.federate((bob, alice))
federated_train_loader = sy.FederatedDataLoader(federated_train_dataset, batch_size=args.batch_size, shuffle=True, **kwargs)


validation_data = datasets.ImageFolder(valid_dir, transform=validation_transforms)

test_loader = torch.utils.data.DataLoader(
  validation_data, batch_size=args.test_batch_size, shuffle=True, **kwargs
)

In [0]:
model = models.densenet169(pretrained=True)

In [0]:
# Freeze the features part
for param in model.parameters():
    param.requires_grad = True
    
# Build our own classifier
# We will be using a sequential model,
# In sequential model we will give it list of different operation
# and it will pass in tensors automatically sequentially.


num_features = 1664 # See the classifier part in the printed model above, it consist in_features=1664


classifier = nn.Sequential(OrderedDict([
                              ('fc1', nn.Linear(num_features, 512)),
                              ('relu_1', nn.ReLU()),
                              ('drpot', nn.Dropout(p=0.5)),
                              ('hidden', nn.Linear(512, 100)),
                              ('relu_2', nn.ReLU()),
                              ('drpot_2', nn.Dropout(p=0.5)),
                              ('fc2', nn.Linear(100, 7)),
                              ('output', nn.LogSoftmax(dim=1)),
                              ]))

model.classifier = classifier


In [0]:

# Move our model from CPU to GPU
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [0]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

In [0]:


def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.step()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))


            

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



In [0]:


%%time

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
  model.class_to_idx = train_loaders.dataset.class_to_idx
  checkpoint = {'input_size': [3, 224, 224],
                 'batch_size': train_loaders.batch_size,
                  'output_size': 7,
                  'state_dict': model.state_dict(),
                  'optimizer_dict':optimizer.state_dict(),
                  'class_to_idx': model.class_to_idx,
               }
  torch.save(checkpoint, "cash_recognition.pt")



In [0]:
# Loading a trained model
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = models.densenet169(pretrained=False)
    
    classifier = nn.Sequential(OrderedDict([
                              ('fc1', nn.Linear(1664, 512)),
                              ('relu_1', nn.ReLU()),
                              ('drpot', nn.Dropout(p=0.5)),
                              ('hidden', nn.Linear(512, 100)),
                              ('relu_2', nn.ReLU()),
                              ('drpot_2', nn.Dropout(p=0.5)),
                              ('fc2', nn.Linear(100, 7)),
                              ('output', nn.LogSoftmax(dim=1)),
                              ]))
    
    model.classifier = classifier
    model.load_state_dict(checkpoint['state_dict'])
    
    return model, checkpoint['class_to_idx']

# class_to_idx will be used later in predicting section.

In [0]:

loaded_model, class_to_idx = load_checkpoint('cash_recognition.pt')
idx_to_class = { v : k for k,v in class_to_idx.items()}


In [0]:
def process_image(image):

    size = 256, 256
    image.thumbnail(size, Image.ANTIALIAS)
    image = image.crop((128 - 112, 128 - 112, 128 + 112, 128 + 112))
    npImage = np.array(image)
    npImage = npImage/255.
        
    imgA = npImage[:,:,0]
    imgB = npImage[:,:,1]
    imgC = npImage[:,:,2]
    
    imgA = (imgA - 0.485)/(0.229) 
    imgB = (imgB - 0.456)/(0.224)
    imgC = (imgC - 0.406)/(0.225)
        
    npImage[:,:,0] = imgA
    npImage[:,:,1] = imgB
    npImage[:,:,2] = imgC
    
    npImage = np.transpose(npImage, (2,0,1))
    
    return npImage

In [0]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [0]:
def predict(image_path, model, topk=5):
    
    image = torch.FloatTensor([process_image(Image.open(image_path))])
    model.eval()
    output = model.forward(Variable(image))
    pobabilities = torch.exp(output).data.numpy()[0]
    

    top_idx = np.argsort(pobabilities)[-topk:][::-1] 
    top_class = [idx_to_class[x] for x in top_idx]
    top_probability = pobabilities[top_idx]

    return top_probability, top_class

In [0]:
def view_classify(img, probabilities, classes, mapper):
    ''' Function for viewing an image and it's predicted classes.
    '''
    img_filename = img.split('/')[-2]
    img = Image.open(img)

    fig, (ax1, ax2) = plt.subplots(figsize=(6,10), ncols=1, nrows=2)
    cash_name = mapper[img_filename]
    
    ax1.set_title(cash_name)
    ax1.imshow(img)
    ax1.axis('off')
    
    y_pos = np.arange(len(probabilities))
    ax2.barh(y_pos, probabilities)
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels([mapper[x] for x in classes])
    ax2.invert_yaxis()

In [0]:
cat_to_name = {
    "fifty": "Fifty",
    "five" : "Five",
    "fivehundred": "Five_Hundred",
    "hundred":"Hundred",
    "ten": "Ten",
    "thousand":"Thousand",
    "twenty": "Twenty"
}

In [0]:
img = 'full_data/valid/thousand/thousand_valid_21.jpg'
p, c = predict(img, loaded_model)
view_classify(img, p, c, cat_to_name)

In [0]:
img = 'full_data/valid/fifty/fifty_valid_91.jpg'
p, c = predict(img, loaded_model)
view_classify(img, p, c, cat_to_name)